<a href="https://colab.research.google.com/github/torzdf/faceswap/blob/colab/faceswap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="left">
  <a href="https://faceswap.dev"><img src="https://i.imgur.com/zHvjHnb.png"></img></a>Welcome to the FaceSwap Notebook
</p>



The FaceSwap Notebook is a means of running FaceSwap on Google Colab.

Colab will give you a free GPU for up to a 12 hour period (as long as your browser window remains open).

Please make sure you understand Google Colaboratory's terms of service. This notebook is for testing out Faceswap and experimenting. It is not for creating swaps on an industrial scale. [See here for more info](https://research.google.com/colaboratory/faq.html#gpu-availability).

At a minimum you must run the [Prerequisites](#scrollTo=GVzyxw4xkKI7). Once complete, you can select the task that you wish to run.

**NB:** This notebook is provided as a courtesy. It is not a priority for the FaceSwap project so there is no guarantee that it will remain up to date and support is likely to be limited. Please be aware that this Notebook lacks some features from the full application.

>[Prerequisites](#scrollTo=GVzyxw4xkKI7)

>>[GPU Check](#scrollTo=_jYRKp5zkYMU)

>>[Link Google Drive](#scrollTo=Sb1aWvH3phjA)

>>[Get FaceSwap](#scrollTo=oql-3Tf7lquV)

>[Run Faceswap](#scrollTo=h9DWTBSHOwxO)

>>[Execute FaceSwap Task](#scrollTo=ffaXw9B5LHNg)



# Prerequisites
First up we need to make sure we have a GPU instance and set up our environment.

Whatever task you plan to perform, you must run these steps first.

## GPU Check
Run the following code block to make sure you have been allocated a GPU:

In [14]:
#@title
!nvidia-smi

Thu Dec 12 01:46:43 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

 You should receive output similar to below:
```
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU       PID   Type   Process name                             Usage      |
|=============================================================================|
|  No running processes found                                                 |
+-----------------------------------------------------------------------------+
```

If you have not been allocated a GPU then select `Runtime` > `Change runtime type` from the top menu and ensure that the `Runtime type` is `Python 3` and `Hardware Acceleration` is `GPU`

## Link Google Drive
Now we know we have a GPU instance, we need to link your Google Drive.

You will need to give Colab permissions to mount your drive, so run the code block below and follow the instructions to mount your drive:

In [1]:
#@title
import os
_ROOT="/content/drive/My Drive/_faceswap_colab"
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

if not os.path.exists(_ROOT):
  !mkdir "$_ROOT"

print("Complete. If you received `shell-init` errors, Try running the cell "
      "again. If you still receive errors; select `Runtime` > `"
      "Restart Runtime... from the main menu and try again.") 
print("\033[32mGoogle Drive Linked\033[0m")


Mounted at /content/drive/
Complete. If you received `shell-init` errors, Try running the cell again. If you still receive errors; select `Runtime` > `Restart Runtime... from the main menu and try again.
Google Drive Linked


## Get FaceSwap
Time to tell Colab where to find/install the FaceSwap App. Select your options in the form, then hit play:

In [2]:
#@title App install location
#@markdown Where do you want Faceswap installed?
#@markdown * GoogleDrive: [Recommended] - Store the FaceSwap App on your Google
#@markdown Drive. This means any config files that are changed won't get lost
#@markdown when the instance destroys. However it will take up about 200MB of
#@markdown space for the base app and several gigs for the associated models.
#@markdown * Local: FaceSwap App will be stored in the Colab Notebook. The app
#@markdown will disappear when the instance dies.
location = "GoogleDrive"  #@param ['GoogleDrive', 'Local']
#@markdown Do You want to update an existing app or create a new one?
#@markdown * Update: [Recommended] - If the FaceSwap App is already installed in
#@markdown the chosen location, it will be updated. If the app doesn't already
#@markdown exist, it will be installed.
#@markdown * New: Uninstall any installed FaceSwap App and Reinstall for the
#@markdown chosen location.
app_update = "Update"  #@param ['New', 'Update']

import os
import shutil
import sys

try:
  _ROOT="/content/drive/My Drive/_faceswap_colab" if location == "GoogleDrive" else "."
  _APPDIR=os.path.join(_ROOT, "faceswap")
  _APP=os.path.join(_APPDIR, "faceswap.py")
  fs_backend_path=os.path.join(_APPDIR, "config", ".faceswap")

  print("\033[32mInstalling Tensorfow...\033[0m")
  !pip install tensorflow-gpu==1.15

  print("\033[32mApp Location:", _APP)

  # Dummy in a backend config in root dir to prevent backend selector popping
  sys_location = os.path.dirname(os.path.realpath(sys.argv[0]))
  sys_conf = os.path.join(sys_location, "config", ".faceswap")
  if not os.path.isdir(os.path.dirname(sys_conf)):
    os.mkdir(os.path.dirname(sys_conf))
    with open(sys_conf, "w") as sys_config:
      sys_config.write("{\"backend\": \"nvidia\"}")

  if app_update == "New" or not os.path.isfile(_APP):

    if os.path.isdir(_APPDIR):
      print("Uninstalling pre-existing app...")
      shutil.rmtree(_APPDIR)

    print("Getting Faceswap...")
    !echo -en "\e[0m"
    !git clone --depth 1 -b master https://github.com/deepfakes/faceswap.git "$_APPDIR"
    %cd "$_APPDIR"

    print("\033[32mInstalling Dependencies. This may take a minute or two...\033[0m")
    !pip install -r "$_APPDIR/requirements.txt"

    print("\033[32mSetting backend to nvidia:\033[0m", fs_backend_path)
    with open(fs_backend_path, "w") as backend_config:
      backend_config.write("{\"backend\": \"nvidia\"}")
  else:
    %cd "$_APPDIR"
    
    print("\033[32mUpdating App...\033[0m")
    !git pull 2>&1

    print("\033[32mSetting backend to nvidia:\033[0m", fs_backend_path)
    with open(fs_backend_path, "w") as backend_config:
      backend_config.write("{\"backend\": \"nvidia\"}")

    print("\033[32mUpdating Dependencies. This may take a minute or two...\033[0m")
    !python "$_APPDIR/update_deps.py"

  print("\033[32mFaceswap Ready!\033[0m")
except OSError as err:
  print("\033[31mYour GDrive could not be reached. Select `Runtime` > `"
      "Restart Runtime... from the main menu and go back to the previous "
      "script (Link Google Drive)")


Installing Tensorfow...
App Location: /content/drive/My Drive/_faceswap_colab/faceswap/faceswap.py
/content/drive/My Drive/_faceswap_colab/faceswap
Updating App...
Already up to date.
Setting backend to nvidia: /content/drive/My Drive/_faceswap_colab/faceswap/config/.faceswap
Updating Dependencies. This may take a minute or two...
INFO    Updating dependencies...
INFO    Setup in Linux 4.14.137+
INFO    Installed Python: 3.6.9 64bit
INFO    Encoding: UTF-8
INFO    Upgrading pip...
INFO    Installed pip: 19.3.1
INFO    Installing Required Python Packages. This may take some time...
INFO    Installing git+https://github.com/deepfakes/nvidia-ml-py3.git
  Cloning https://github.com/deepfakes/nvidia-ml-py3.git to /tmp/pip-req-build-hwe6wwoe
  Running command git clone -q https://github.com/deepfakes/nvidia-ml-py3.git /tmp/pip-req-build-hwe6wwoe
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.1-cp36-none-any.whl size=19268 sha256=3b3b9537d43cd319f84dcae4715ed87365d6f716a2c40c

# Run Faceswap
Now we're all setup, it's time to choose a task.

Remember, you can edit the plugin configuration files here before proceeding:
`/drive/My Drive/_faceswap_colab/faceswap/config`

Select your option in the form, then hit play:

In [3]:
#@title Configure FaceSwap Task
#@markdown Which task do you want to run?
task = "Train"  #@param ['Extract', 'Train', 'Convert']
#@markdown Once you're happy with your selection, run this code block to
#@markdown generate the options form for the selected Task.
#@markdown If required, you can change task and re-run this code block to
#@markdown generate options for a different Task.

#@markdown Note: You can copy file paths from the `Files` tab of the lefthand
#@markdown panel by right clicking the desired file and selecting `Copy path`.
#@markdown Your GoogleDrive is located in `/drive/My Drive`.

from textwrap import wrap
from ipywidgets import widgets
import inspect
def get_cliopts(section):
  """ obtain the cli options for the given section """
  import lib.cli as cli
  mod_class=[name for name, obj in inspect.getmembers(cli)
             if inspect.isclass(obj) and name.lower().endswith("args")
             and name.lower().startswith(section)
             and name.lower() != "extractconvertargs"][0]

  name = mod_class.lower().replace("args", "")
  meth = getattr(cli, mod_class)(None, name)
  print("\033[32m{}\033[0m".format(name.title()))
  print(meth.info.split("\n")[0], "\n")

  browser_lookup = dict(DirFullPaths="Folder",
                        FileFullPaths="File",
                        DirOrFileFullPaths="Folder or a File")
  args = meth.argument_list + meth.optional_arguments + meth.global_arguments
  final_args = []
  # Intermediate color for light and dark skin
  for arg in args:
    action = arg.get("action", None)
    action = action.__name__ if inspect.isclass(action) else action
    _opt_name = arg["opts"][-1].replace("--", "")
    if _opt_name in ("logfile", "gui", "singleprocess", "colab", "preview"):
      continue
    val = arg.get("default", None)
    if action == "Radio":
      arg["control"] = widgets.RadioButtons(
          options=arg.get("choices", []),
          value=val,
          disabled=False)
    elif action == "Slider" and arg["type"] == int:
      arg["control"] = widgets.IntSlider(
        value=val,
        min=arg["min_max"][0],
        max=arg["min_max"][1],
        step=arg["rounding"],
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',
        options=arg.get("choices", []))
    elif action == "Slider" and arg["type"] == float:
      arg["control"] = widgets.FloatSlider(
        value=val,
        min=arg["min_max"][0],
        max=arg["min_max"][1],
        step=arg["rounding"] / 100,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f')
    elif action == "store_true":
      arg["control"] = widgets.Checkbox(
        value=val,
        disabled=False)
    elif action in browser_lookup:
        arg["control"] = widgets.Text(
        value=val,
        placeholder='Enter the path to a {}'.format(browser_lookup[action]),
        disabled=False)      
    elif arg.get("choices", None) is not None:
      arg["control"] = widgets.Dropdown(
        options=arg["choices"],
        value=val,
        disabled=False)
    elif arg.get("type", str) == int:
      arg["control"] = widgets.IntText(
        value=val,
        disabled=False)
    elif arg.get("type", str) == float:
      arg["control"] = widgets.FloatText(
        value=val,
        disabled=False)
    else:
      arg["control"] = widgets.Text(
        value=val,
        placeholder='Type something',
        disabled=False)      
    final_args.append(arg)
  return final_args

_cli_opts = get_cliopts(task.lower())
_groups = dict()

for opt in _cli_opts:
  _groups.setdefault(opt.get("group", "").title(), []).append(opt)

for group, opts in _groups.items():
  print("\033[31m{}\n== {}\n{}\033[0m".format("=" * 40, group, "=" * 40))
  for opt in opts:
    _opt_name = opt["opts"][-1].replace("--", "")
    print("-- \033[34m{}\033[0m {}".format(_opt_name, "-" * 100)[:109])
    display(opt["control"])

    _hlp = opt.get("help", "")
    if _hlp.startswith("R|"):
      line = _hlp[2:].replace("L|", "L|  - ")
      for row in line.split("L|"):
        for outp in wrap(row, 100):
          print(outp)
    else: 
      for line in wrap(opt.get("help", ""), 100):
        print(line)
    print("{}\n".format("-" * 100))


Setting Faceswap backend to NVIDIA
Train
Train a model on extracted original (A) and swap (B) faces. 

== Faces
-- input-A -----------------------------------------------------------------------------------------


Text(value='', placeholder='Enter the path to a Folder')

Input directory. A directory containing training images for face A. This is the original face, i.e.
the face that you want to remove and replace with face B.
----------------------------------------------------------------------------------------------------

-- alignments-A ------------------------------------------------------------------------------------


Text(value='', placeholder='Enter the path to a File')

Path to alignments file for training set A. Only required if you are using a masked model or warp-
to-landmarks is enabled. Defaults to <input-A>/alignments.json if not provided.
----------------------------------------------------------------------------------------------------

-- input-B -----------------------------------------------------------------------------------------


...

Writes the training result to a file. The image will be stored in the root of your FaceSwap folder.
----------------------------------------------------------------------------------------------------

== Global Options
-- configfile --------------------------------------------------------------------------------------


Text(value='', placeholder='Enter the path to a File')

Optionally overide the saved config with the path to a custom config file.
----------------------------------------------------------------------------------------------------

-- loglevel ----------------------------------------------------------------------------------------


Dropdown(options=('INFO', 'VERBOSE', 'DEBUG', 'TRACE'), value='INFO')

Log level. Stick with INFO or VERBOSE unless you need to file an error report. Be careful with TRACE
as it will generate a lot of data
----------------------------------------------------------------------------------------------------



## Execute FaceSwap Task
If you're happy with your options, then it's time to run the Task. Run the codeblock below to generate the task:

In [21]:
#@title
_exe_args = task.lower()
for opt in _cli_opts:
  val = opt["control"].value
  if not val:
    continue
  _exe_args += " {}".format(opt["opts"][0])
  if opt.get("action", None) != "store_true":
    val = "\"{}\"".format(val) if isinstance(val, str) and " " in val else val
    _exe_args += " {}".format(val)

print("The command to be run is as follows:\n\n\033[34m{}\033[0m\n\n"
      "Click the 'Execute' button to proceed, or update your options and "
      "re-run this code block\n".format(_exe_args))

button = widgets.Button(description="Execute {}!".format(task))
output = widgets.Output()

def on_button_clicked(b):
  print("\nExecuting Faceswap...")
  !echo "$_exe_args --colab"
  !python faceswap.py $_exe_args

button.on_click(on_button_clicked)
display(button, output)

  

The command to be run is as follows:

extract -D s3fd -A fan -M extended -nm none -l 0.4 -een 1 -sz 256 -L INFO

Click the 'Execute' button to proceed, or update your options and re-run this code block



Button(description='Execute Extract!', style=ButtonStyle())

Output()